In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [2]:
names = ['Customer_id','Gender','Booking_date','Booking_time','Pickup_address','Pickup_time','Drop_time','Drop_address','Cost','Month','Date','Hour']

In [3]:
df = pd.read_csv('s3://deloitte-case-study-2-group-3-processed/redshift_downloaded_data/000',delimiter='|',names=names)

In [4]:
df.head()

,Customer_id,Gender,Booking_date,Booking_time,Pickup_address,Pickup_time,Drop_time,Drop_address,Cost,Month,Date,Hour
0,542,female,2022-04-01,07:33:28.358151,"9545 Christopher Forges Apt. 866 New Sarah, SD...",07:38:28.358152,09:02:28.358152,48025 Michael Mountains Suite 257 North Donald...,300,4,1,7
1,1000,male,2022-04-01,07:33:58.569815,13443 Kristen Meadow Suite 384 New Ashleymouth...,07:39:58.569817,09:29:58.569817,37100 Dominic Ports Suite 903 North Georgeside...,331,4,1,7
2,733,female,2022-04-01,07:33:48.496508,68374 Anderson Harbor Apt. 037 North Victoriaf...,07:43:48.496510,09:40:48.496510,"6109 Mckinney Light Apt. 875 Jamesburgh, DE 04041",385,4,1,7
3,315,male,2022-04-01,07:33:48.511281,0412 Donna Stravenue Apt. 818 North Timothymou...,07:42:48.511282,09:32:48.511282,"894 Cesar Extensions Apt. 008 North Travis, WV...",365,4,1,7
4,590,female,2022-04-01,07:33:58.594846,657 Melissa Circles Suite 098 New Jamesborough...,07:43:58.594848,08:36:58.594848,"47898 Kevin Alley Apt. 703 East Joshuastad, NJ...",213,4,1,7


In [5]:
df.isnull().sum()

Customer_id       0
Gender            0
Booking_date      0
Booking_time      0
Pickup_address    0
Pickup_time       0
Drop_time         0
Drop_address      0
Cost              0
Month             0
Date              0
Hour              0
dtype: int64

In [6]:
df.dtypes

Customer_id        int64
Gender            object
Booking_date      object
Booking_time      object
Pickup_address    object
Pickup_time       object
Drop_time         object
Drop_address      object
Cost               int64
Month              int64
Date               int64
Hour               int64
dtype: object

In [7]:
df.shape

(61000, 12)

In [8]:
import datetime


In [9]:
df['Booking_time'][0].split(',')[0]

'07:33:28.358151'

In [10]:
def convert_to_time(col_val):
    return datetime.datetime.strptime(col_val.split('.')[0], '%H:%M:%S')
    

In [11]:
df['Booking_time_dt'] = df['Booking_time'].apply(convert_to_time)
df['Pickup_time_dt'] = df['Pickup_time'].apply(convert_to_time)
df['Drop_time_dt'] = df['Drop_time'].apply(convert_to_time)

In [12]:
type(df['Booking_time_dt'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [13]:
df['book_vs_pick'] = (df['Pickup_time_dt'] - df['Booking_time_dt'])
df['pick_vs_drop'] = (df['Drop_time_dt'] - df['Pickup_time_dt'])

In [14]:
def conv_to_min(x):
    return x.total_seconds()/60

In [15]:
df['pick_time_in_min'] = df['book_vs_pick'].apply(conv_to_min)
df['travel_time_in_min'] = df['pick_vs_drop'].apply(conv_to_min)


In [16]:
df.head()

,Customer_id,Gender,Booking_date,Booking_time,Pickup_address,Pickup_time,Drop_time,Drop_address,Cost,Month,Date,Hour,Booking_time_dt,Pickup_time_dt,Drop_time_dt,book_vs_pick,pick_vs_drop,pick_time_in_min,travel_time_in_min
0,542,female,2022-04-01,07:33:28.358151,"9545 Christopher Forges Apt. 866 New Sarah, SD...",07:38:28.358152,09:02:28.358152,48025 Michael Mountains Suite 257 North Donald...,300,4,1,7,1900-01-01 07:33:28,1900-01-01 07:38:28,1900-01-01 09:02:28,0 days 00:05:00,0 days 01:24:00,5.0,84.0
1,1000,male,2022-04-01,07:33:58.569815,13443 Kristen Meadow Suite 384 New Ashleymouth...,07:39:58.569817,09:29:58.569817,37100 Dominic Ports Suite 903 North Georgeside...,331,4,1,7,1900-01-01 07:33:58,1900-01-01 07:39:58,1900-01-01 09:29:58,0 days 00:06:00,0 days 01:50:00,6.0,110.0
2,733,female,2022-04-01,07:33:48.496508,68374 Anderson Harbor Apt. 037 North Victoriaf...,07:43:48.496510,09:40:48.496510,"6109 Mckinney Light Apt. 875 Jamesburgh, DE 04041",385,4,1,7,1900-01-01 07:33:48,1900-01-01 07:43:48,1900-01-01 09:40:48,0 days 00:10:00,0 days 01:57:00,10.0,117.0
3,315,male,2022-04-01,07:33:48.511281,0412 Donna Stravenue Apt. 818 North Timothymou...,07:42:48.511282,09:32:48.511282,"894 Cesar Extensions Apt. 008 North Travis, WV...",365,4,1,7,1900-01-01 07:33:48,1900-01-01 07:42:48,1900-01-01 09:32:48,0 days 00:09:00,0 days 01:50:00,9.0,110.0
4,590,female,2022-04-01,07:33:58.594846,657 Melissa Circles Suite 098 New Jamesborough...,07:43:58.594848,08:36:58.594848,"47898 Kevin Alley Apt. 703 East Joshuastad, NJ...",213,4,1,7,1900-01-01 07:33:58,1900-01-01 07:43:58,1900-01-01 08:36:58,0 days 00:10:00,0 days 00:53:00,10.0,53.0


In [17]:
df.columns

Index(['Customer_id', 'Gender', 'Booking_date', 'Booking_time',
       'Pickup_address', 'Pickup_time', 'Drop_time', 'Drop_address', 'Cost',
       'Month', 'Date', 'Hour', 'Booking_time_dt', 'Pickup_time_dt',
       'Drop_time_dt', 'book_vs_pick', 'pick_vs_drop', 'pick_time_in_min',
       'travel_time_in_min'],
      dtype='object')

In [18]:
df_final = df.drop(['Booking_time','Pickup_time','Drop_time','Booking_time_dt', 'Pickup_time_dt',
       'Drop_time_dt', 'book_vs_pick', 'pick_vs_drop'],axis=1)

In [19]:
df_final.head()

,Customer_id,Gender,Booking_date,Pickup_address,Drop_address,Cost,Month,Date,Hour,pick_time_in_min,travel_time_in_min
0,542,female,2022-04-01,"9545 Christopher Forges Apt. 866 New Sarah, SD...",48025 Michael Mountains Suite 257 North Donald...,300,4,1,7,5.0,84.0
1,1000,male,2022-04-01,13443 Kristen Meadow Suite 384 New Ashleymouth...,37100 Dominic Ports Suite 903 North Georgeside...,331,4,1,7,6.0,110.0
2,733,female,2022-04-01,68374 Anderson Harbor Apt. 037 North Victoriaf...,"6109 Mckinney Light Apt. 875 Jamesburgh, DE 04041",385,4,1,7,10.0,117.0
3,315,male,2022-04-01,0412 Donna Stravenue Apt. 818 North Timothymou...,"894 Cesar Extensions Apt. 008 North Travis, WV...",365,4,1,7,9.0,110.0
4,590,female,2022-04-01,657 Melissa Circles Suite 098 New Jamesborough...,"47898 Kevin Alley Apt. 703 East Joshuastad, NJ...",213,4,1,7,10.0,53.0


In [39]:


bins = [4,7,9,11,13,15]

slots = ["5 to 7 mins","7 to 9 mins","9 to 11 mins","11 to 13 mins","13 to 15 mins"]

df_final['pick_time_cat'] = pd.cut(df_final['pick_time_in_min'], bins=bins, labels=slots)

In [40]:
df_final.head()

,Customer_id,Gender,Booking_date,Pickup_address,Drop_address,Cost,Month,Date,Hour,pick_time_in_min,travel_time_in_min,pick_time_cat
0,542,0,2022-04-01,"9545 Christopher Forges Apt. 866 New Sarah, SD...",48025 Michael Mountains Suite 257 North Donald...,300,4,1,7,5.0,84.0,5 to 7 mins
1,1000,1,2022-04-01,13443 Kristen Meadow Suite 384 New Ashleymouth...,37100 Dominic Ports Suite 903 North Georgeside...,331,4,1,7,6.0,110.0,5 to 7 mins
2,733,0,2022-04-01,68374 Anderson Harbor Apt. 037 North Victoriaf...,"6109 Mckinney Light Apt. 875 Jamesburgh, DE 04041",385,4,1,7,10.0,117.0,9 to 11 mins
3,315,1,2022-04-01,0412 Donna Stravenue Apt. 818 North Timothymou...,"894 Cesar Extensions Apt. 008 North Travis, WV...",365,4,1,7,9.0,110.0,7 to 9 mins
4,590,0,2022-04-01,657 Melissa Circles Suite 098 New Jamesborough...,"47898 Kevin Alley Apt. 703 East Joshuastad, NJ...",213,4,1,7,10.0,53.0,9 to 11 mins


In [42]:
# df_final.isnull().sum()

In [20]:
df_final.to_csv('final_table.csv')

In [43]:
bins = [14,30,45,60,75,90,105,120]

slots = ["15 to 30 mins","40 to 45 mins","45 to 60 mins","60 to 75 mins","75 to 90 mins","90 to 105 mins","105 to 120 mins"]

df_final['travel_time_cat'] = pd.cut(df_final['travel_time_in_min'], bins=bins, labels=slots)

In [44]:
df_final.head()

,Customer_id,Gender,Booking_date,Pickup_address,Drop_address,Cost,Month,Date,Hour,pick_time_in_min,travel_time_in_min,pick_time_cat,travel_time_cat
0,542,0,2022-04-01,"9545 Christopher Forges Apt. 866 New Sarah, SD...",48025 Michael Mountains Suite 257 North Donald...,300,4,1,7,5.0,84.0,5 to 7 mins,75 to 90 mins
1,1000,1,2022-04-01,13443 Kristen Meadow Suite 384 New Ashleymouth...,37100 Dominic Ports Suite 903 North Georgeside...,331,4,1,7,6.0,110.0,5 to 7 mins,105 to 120 mins
2,733,0,2022-04-01,68374 Anderson Harbor Apt. 037 North Victoriaf...,"6109 Mckinney Light Apt. 875 Jamesburgh, DE 04041",385,4,1,7,10.0,117.0,9 to 11 mins,105 to 120 mins
3,315,1,2022-04-01,0412 Donna Stravenue Apt. 818 North Timothymou...,"894 Cesar Extensions Apt. 008 North Travis, WV...",365,4,1,7,9.0,110.0,7 to 9 mins,105 to 120 mins
4,590,0,2022-04-01,657 Melissa Circles Suite 098 New Jamesborough...,"47898 Kevin Alley Apt. 703 East Joshuastad, NJ...",213,4,1,7,10.0,53.0,9 to 11 mins,45 to 60 mins


In [45]:
import boto3

s3 = boto3.resource('s3')
BUCKET = "deloitte-case-study-2-group-3-processed"

s3.Bucket(BUCKET).upload_file("final_table.csv", "final_table_for_visual.csv")

In [23]:
gender_map = {
    'female':0,
    'male':1
}

In [24]:
df_final['Gender'] = df_final['Gender'].map(gender_map)

In [25]:
df_final.head()

,Customer_id,Gender,Booking_date,Pickup_address,Drop_address,Cost,Month,Date,Hour,pick_time_in_min,travel_time_in_min
0,542,0,2022-04-01,"9545 Christopher Forges Apt. 866 New Sarah, SD...",48025 Michael Mountains Suite 257 North Donald...,300,4,1,7,5.0,84.0
1,1000,1,2022-04-01,13443 Kristen Meadow Suite 384 New Ashleymouth...,37100 Dominic Ports Suite 903 North Georgeside...,331,4,1,7,6.0,110.0
2,733,0,2022-04-01,68374 Anderson Harbor Apt. 037 North Victoriaf...,"6109 Mckinney Light Apt. 875 Jamesburgh, DE 04041",385,4,1,7,10.0,117.0
3,315,1,2022-04-01,0412 Donna Stravenue Apt. 818 North Timothymou...,"894 Cesar Extensions Apt. 008 North Travis, WV...",365,4,1,7,9.0,110.0
4,590,0,2022-04-01,657 Melissa Circles Suite 098 New Jamesborough...,"47898 Kevin Alley Apt. 703 East Joshuastad, NJ...",213,4,1,7,10.0,53.0


In [26]:
df_final['Hour'].unique()

array([7, 8])

In [30]:
df_final.shape

(61000, 11)

In [33]:
loc_dict = {}
count = 0
for i in df_final['Pickup_address']:
    if i not in loc_dict.keys():
        loc_dict[i] = count
        count += 1
for i in df_final['Drop_address']:
    if i not in loc_dict.keys():
        loc_dict[i] = count
        count += 1
        

In [34]:
len(loc_dict)

122000

In [47]:
input_cols = df_final[['Gender','Hour','pick_time_in_min','travel_time_in_min']]

In [48]:
out_col = df_final['Cost']

In [51]:
# test train split

input_train, input_test, output_train, output_test = train_test_split(input_cols,out_col,test_size=0.2)

In [52]:
def model_evaluate(model):
    m = model()
    m.fit(input_train,output_train)
    pred = m.predict(input_test)
    return m,mean_absolute_error(pred,output_test),mean_squared_error(pred,output_test),r2_score(pred,output_test)

In [53]:
li = model_evaluate(LinearRegression)

In [54]:
li

(LinearRegression(), 13.75248864142626, 251.53920974648886, 0.9690847162596475)

In [55]:
rid = model_evaluate(Ridge)

In [56]:
rid

(Ridge(), 13.752488635942978, 251.5392120413536, 0.9690847146162704)

In [57]:
las  = model_evaluate(Lasso)

In [58]:
las

(Lasso(), 13.75259999506322, 251.6140022178343, 0.9690530295638085)

In [59]:
import pickle

In [61]:
pickle.dump(li[0],open('model.sav','wb'))

In [62]:
li[0].predict([[0,7,6,80]])

array([267.89486683])

In [63]:
# gender , time , picktime , travel time